## 🤖 Simulación de un Bot de Trading con Eventos Discretos

Este notebook tiene como objetivo explorar y analizar algunos comportamientos de bots de trading mediante simulaciones de eventos discretos. 

Sientace libre de crear nuevos analisis y jugar con los parametros de la simulacion para obtener nuevos resultados, ayudandote del codigo que se brinda en el proyecto


## 🛠️ Importación de librerías y funciones

In [5]:
import sys
sys.path.append('./src')
from src.bot import TradingBot
from src.market import *
from src.analysis import *
from src.plotter import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## 📦Variables Globales

In [ ]:
bot_conservador = {"name": "Conservador", "initial_cash": 1000, "buy_thresh": 0.98, "sell_thresh": 1.02, "fee": 0.005}
bot_agresivo = {"name": "Agresivo", "initial_cash": 1000, "buy_thresh": 0.95, "sell_thresh": 1.05, "fee": 0.005}
bot_tendencia = {"name": "Tendencia", "initial_cash": 10000, "buy_thresh": 0.97, "sell_thresh": 1.03, "fee": 0.005}
gwt_params = {'n_steps':300, 'initial_price':100, 'sigma':1.0, 'mu':0.1, 'seed':None}
bot_config = bot_tendencia
generators = {"Gaussiano": gaussian_walk,"Con Tendencia": gaussian_walk_with_trend,"Geom Brown": geometric_brownian_motion,"Mean Revert": mean_reverting}

## 🔍 Análisis 1: Comparación de estrategias (Conservador vs Agresivo)

**Objetivo:** Comparar dos estrategias de bot para evaluar cuál obtiene mayor ganancia media y menor varianza.

In [ ]:
df_conservador = simulate_multiple_runs(bot_conservador, n_runs=10000)
df_agresivo = simulate_multiple_runs(bot_agresivo, n_runs=10000)

resumen = pd.DataFrame({
    "Bot": ["Conservador", "Agresivo"],
    "Ganancia Prom.": [df_conservador["net_worth"].mean(), df_agresivo["net_worth"].mean()],
    "Trades Prom.": [df_conservador["trades"].mean(), df_agresivo["trades"].mean()],
    "Std Dev": [df_conservador["net_worth"].std(), df_agresivo["net_worth"].std()]
})
resumen

## **Interpretación:**

El bot agresivo obtiene mayor ganancia promedio, pero su desviación estándar también es más alta. Esto implica mayor riesgo, lo cual puede ser deseable o no según el contexto del mercado.

## Mostrar y Guardar los graficos

In [ ]:
plot_distribution(df_conservador, save_path="figures/distribucion_bot.png", title="bot_conservador")
stats = test_normality(df_conservador)
print("Shapiro:", stats["shapiro"])
print("D'Agostino:", stats["normaltest"], '\n')

df_conservador.to_csv("data/resultados_bot_conservador.csv", index=False)


plot_distribution(df_agresivo, save_path="figures/distribucion_bot.png", title="bot_agresivo")
stats = test_normality(df_agresivo)
print("Shapiro:", stats["shapiro"])
print("D'Agostino:", stats["normaltest"])

df_agresivo.to_csv("data/resultados_bot_agresivo.csv", index=False)


## 🔍 Análisis 2: ¿Influye la tendencia del mercado en el desempeño del bot?

**Objetivo:** Comparar el comportamiento del bot en mercados con y sin tendencia.


In [ ]:
resultados = {}

for nombre, gen_func in generators.items():
    df = simulate_multiple_runs(bot_config, n_runs=100, n_steps=300,
                                sigma=1.0 if nombre != "Geom Brown" else 0.02)
    resultados[nombre] = df
    print(f"🧪 {nombre}: media net worth = ${df['net_worth'].mean():.2f}")


print("Comparación entre Gaussiano y Mean-Reverting:")
result = compare_bots(resultados["Gaussiano"], resultados["Mean Revert"])
for name, (stat, p) in result.items():
    print(f"{name:>15}: stat={stat:.4f}, p-value={p:.4e}")

plot_price_model_comparison(
    resultados_dict=resultados,  # Este diccionario contiene los DataFrames por modelo
    metric="net_worth",
    save_path="figures/comparacion_modelos_networth.png",
    title="Ganancia Final según el Modelo de Precios"
)

plot_price_model_comparison(
    resultados_dict=resultados,
    metric="trades",
    save_path="figures/comparacion_modelos_trades.png",
    title="Cantidad de Operaciones según el Modelo de Precios"
)

## **Interpretación:**



## 🔍 Análisis 3: Estimación de las metricas con Bootstrap

**Objetivo:** Validar y construir un intervalo de confianza para cada una de las metricas del bot.


In [ ]:
df = simulate_multiple_runs(bot_config, n_runs=100)

# Métricas a analizar con bootstrap
metrics = ["net_worth", "trades", "final_cash", "final_position"]

for metric in metrics:
    media, (low, high), samples = bootstrap_confidence_interval(df[metric].values)
    
    print(f"\n📈 Métrica: {metric}")
    print(f"    Media bootstrap: {media:.2f}")
    print(f"    IC 95%: ({low:.2f}, {high:.2f})")

    plot_bootstrap_distribution(
        samples,
        real_value=df[metric].mean(),
        ci=(low, high),
        save_path=f"figures/bootstrap_{metric}.png",
        title=f"Distribución Bootstrap - {metric}"
    )

## 🔍 Análisis 4: ¿Cuántas simulaciones necesito para estimar con confianza cuánto gana mi bot?

### 🎯 Objetivo:

Simular al bot repetidas veces y detenerse automáticamente cuando el intervalo de confianza para la ganancia promedio sea suficientemente pequeño:

In [ ]:
df, final_error, error_trace = simulate_until_confidence(bot_config, d=100)

print(f"[✔] Simulaciones necesarias: {len(df)}")
print(f"[±] Error final estimado: ${final_error:.2f}")

df.to_csv("data/resultados_bot_parada.csv", index=False)
plot_distribution(df, save_path="figures/distribucion_bot_parada.png")

## 🔍 Análisis 5: Comparación entre horizontes (semana, mes, año)
**Objetivo:** Analizar si el desempeño del bot mejora o empeora al cambiar la duración del escenario simulado.


In [ ]:
horizontes = {"Semana": 50, "Mes": 200, "Año": 1000}
resultados_h = {}

for nombre, pasos in horizontes.items():
    df = simulate_multiple_runs(bot_tendencia, n_runs=100, n_steps=pasos)
    resultados_h[nombre] = df

plot_price_model_comparison(resultados_h, metric="net_worth", title="Ganancia según horizonte")
